In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import librosa           
import librosa.display     
import warnings
warnings.filterwarnings('ignore')       


## Extraction of audio features
base_path = Path.cwd()
all_files = base_path/"Data"/"genres_original"
list_of_rows  = []
for genre_folder in all_files.iterdir():
    genre_path = all_files/genre_folder
    genre_name = genre_folder.name
    for song in genre_path.iterdir():
        try:
            song_path = genre_path/song
            y, sr = librosa.load(song_path, sr=None, mono=True) # Mono ensures only 1 signal is accepted so only 1D array is created
            zcr = librosa.feature.zero_crossing_rate(y)
            zcr_mean = np.mean(zcr)
            zcr_sd = np.std(zcr)
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            sc_mean = np.mean(spectral_centroid)
            sc_sd = np.std(spectral_centroid)
            new_row = {'Genre': genre_name ,'ZCR Mean':zcr_mean, 'ZCR STD': zcr_sd, 'Spectral Centroid Mean':sc_mean,
                       'Spectral Centroid STD': sc_sd}
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40,n_fft=256, n_mels=40)
            for i in range(len(mfccs)):
                idx = str(i+1)
                mfcc_mean = np.mean(mfccs[i])
                mfcc_sd = np.std(mfccs[i])
                mean_name = 'MFCC ' + idx + ' MEAN'
                std_name = 'MFCC ' + idx + ' STD'
                new_row[mean_name]=mfcc_mean
                new_row[std_name]=mfcc_sd
            list_of_rows.append(new_row)
        except Exception as e:
            print(e)
            continue

## Creating a dataframe with the extracted features
music_analytics = pd.DataFrame(list_of_rows)
music_analytics.to_csv('features.txt', sep='\t', index=False)


